# Fannie Mae analysis

See for futher details: 
https://www.fanniemae.com/portal/funding-the-market/data/loan-performance-data.html

Per quarter there is a Acquisition data set and a Performance data set. See for details
https://www.fanniemae.com/resources/file/fundmarket/pdf/webinar-101.pdf

In [1]:
import pandas as pd
import datatable as dt

## import acquisition and performance data

In [7]:
%%time
Acquisitions_Variables = [
    "LOAN_ID", "ORIG_CHN", "Seller_Name", "ORIG_RT", "ORIG_AMT", "ORIG_TRM", "ORIG_DTE"
    ,"FRST_DTE", "OLTV", "OCLTV", "NUM_BO", "DTI", "CSCORE_B", "FTHB_FLG", "PURPOSE", "PROP_TYP"
    ,"NUM_UNIT", "OCC_STAT", "STATE", "ZIP_3", "MI_PCT", "Product_Type", "CSCORE_C", "MI_TYPE", "RELOCATION_FLG"
]


AQ2018Q1 = dt.fread(
    "data/Acquisition_2018Q1.txt",
    sep = "|",
    header = None 
)

AQ2018Q1.names = Acquisitions_Variables
AQ2018Q1 = AQ2018Q1.to_pandas()
AQ2018Q1.shape

CPU times: user 3.94 s, sys: 224 ms, total: 4.17 s
Wall time: 815 ms


(426207, 25)

In [3]:
AQ2018Q1.head(5)

,LOAN_ID,ORIG_CHN,Seller_Name,ORIG_RT,ORIG_AMT,ORIG_TRM,ORIG_DTE,FRST_DTE,OLTV,OCLTV,...,PROP_TYP,NUM_UNIT,OCC_STAT,STATE,ZIP_3,MI_PCT,Product_Type,CSCORE_C,MI_TYPE,RELOCATION_FLG
0,100001040173,R,QUICKEN LOANS INC.,4.25,453000,360,01/2018,03/2018,65,65.0,...,PU,1,P,OH,430,NaN,FRM,NaN,NaN,N
1,100002370993,C,"WELLS FARGO BANK, N.A.",4.25,266000,360,01/2018,03/2018,80,80.0,...,PU,1,P,IN,467,NaN,FRM,793.0,NaN,N
2,100005405807,R,PMTT4,3.99,233000,360,12/2017,01/2018,79,79.0,...,SF,1,P,CA,936,NaN,FRM,665.0,NaN,N
3,100008071646,R,OTHER,4.25,184000,360,01/2018,03/2018,80,80.0,...,PU,1,P,FL,336,NaN,FRM,NaN,NaN,N
4,100010739040,R,OTHER,4.25,242000,360,02/2018,04/2018,49,49.0,...,SF,1,P,CA,906,NaN,FRM,NaN,NaN,N


In [6]:
%%time
Performance_Variables = [
    "LOAN_ID", "Monthly_Rpt_Prd", "Servicer_Name", "LAST_RT", "LAST_UPB", "Loan_Age", "Months_To_Legal_Mat",
    "Adj_Month_To_Mat", "Maturity_Date", "MSA", "Delq_Status", "MOD_FLAG", "Zero_Bal_Code", 
    "ZB_DTE", "LPI_DTE", "FCC_DTE","DISP_DT", "FCC_COST", "PP_COST", "AR_COST", "IE_COST", "TAX_COST", "NS_PROCS",
    "CE_PROCS", "RMW_PROCS", "O_PROCS", "NON_INT_UPB", "PRIN_FORG_UPB_FHFA", "REPCH_FLAG", "PRIN_FORG_UPB_OTH", "TRANSFER_FLG"
]

PERF2018Q1 = dt.fread(
    "data/Performance_2018Q1.txt",
    sep = "|",
    header = None 
)

PERF2018Q1.names = Performance_Variables
PERF2018Q1 = PERF2018Q1.to_pandas()
PERF2018Q1.shape

CPU times: user 34.2 s, sys: 5.02 s, total: 39.2 s
Wall time: 14.8 s


(9332050, 31)

In [8]:
PERF2018Q1.head(5)

,LOAN_ID,Monthly_Rpt_Prd,Servicer_Name,LAST_RT,LAST_UPB,Loan_Age,Months_To_Legal_Mat,Adj_Month_To_Mat,Maturity_Date,MSA,...,TAX_COST,NS_PROCS,CE_PROCS,RMW_PROCS,O_PROCS,NON_INT_UPB,PRIN_FORG_UPB_FHFA,REPCH_FLAG,PRIN_FORG_UPB_OTH,TRANSFER_FLG
0,100001040173,02/01/2018,QUICKEN LOANS INC.,4.25,NaN,0,360,360.0,02/2048,18140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,N
1,100001040173,03/01/2018,,4.25,NaN,1,359,359.0,02/2048,18140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,N
2,100001040173,04/01/2018,,4.25,NaN,2,358,358.0,02/2048,18140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,N
3,100001040173,05/01/2018,,4.25,NaN,3,357,357.0,02/2048,18140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,N
4,100001040173,06/01/2018,,4.25,NaN,4,356,356.0,02/2048,18140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,N


In [11]:
test = (
    AQ2018Q1
    .merge(
        PERF2018Q1,
        how="left",
        left_on="LOAN_ID",
        right_on="LOAN_ID"
    )
    .filter([ 
        "LOAN_ID","ORIG_DTE","FRST_DTE",
        "Monthly_Rpt_Prd", "Loan_Age", "Seller_Name", "ORIG_RT",	"ORIG_AMT",
        "Zero_Bal_Code", "Delq_Status", "ZB_DTE", "LPI_DTE"
    ])
)

In [12]:
test

,LOAN_ID,ORIG_DTE,FRST_DTE,Monthly_Rpt_Prd,Loan_Age,Seller_Name,ORIG_RT,ORIG_AMT,Zero_Bal_Code,Delq_Status,ZB_DTE,LPI_DTE
0,100001040173,01/2018,03/2018,02/01/2018,0.0,QUICKEN LOANS INC.,4.25,453000,NaN,0,,
1,100001040173,01/2018,03/2018,03/01/2018,1.0,QUICKEN LOANS INC.,4.25,453000,NaN,0,,
2,100001040173,01/2018,03/2018,04/01/2018,2.0,QUICKEN LOANS INC.,4.25,453000,NaN,0,,
3,100001040173,01/2018,03/2018,05/01/2018,3.0,QUICKEN LOANS INC.,4.25,453000,NaN,0,,
4,100001040173,01/2018,03/2018,06/01/2018,4.0,QUICKEN LOANS INC.,4.25,453000,NaN,0,,
...,...,...,...,...,...,...,...,...,...,...,...,...
9332046,999998783494,01/2018,03/2018,08/01/2019,18.0,"CALIBER HOME LOANS, INC.",4.25,358000,NaN,0,,
9332047,999998783494,01/2018,03/2018,09/01/2019,19.0,"CALIBER HOME LOANS, INC.",4.25,358000,NaN,0,,
9332048,999998783494,01/2018,03/2018,10/01/2019,20.0,"CALIBER HOME LOANS, INC.",4.25,358000,NaN,0,,
9332049,999998783494,01/2018,03/2018,11/01/2019,21.0,"CALIBER HOME LOANS, INC.",4.25,358000,NaN,0,,


The column Delq_Status is the Loan Delinquency status and has the following meaning:

* 0 - "Current or less than 30 days past due"
* 1 - "30 - 59 days past due"
* 2 - "60 - 89 days past due"
* 3 - "90 - 119 days past due"
* 4 - "120 - 149 days past due"
* 5 - "150 - 179 days past due"
* 6 - "180 Day Delinquency"
* 7 - "210 Day Delinquency"
* 8 - "240 Day Delinquency"
* 9 - "270 Day Delinquency" / "270+ Day Delinquency"

In [13]:
test.Delq_Status.value_counts()

0     9187791
X       68114
1       52514
2        9927
3        4143
4        2669
5        2016
6        1336
7         941
8         648
9         446
10        375
11        274
12        199
13        158
          137
14        119
15         96
16         51
17         38
18         24
19         16
20         11
21          5
22          2
Name: Delq_Status, dtype: int64

In [52]:
test.query("Delq_Status == '5'")

,LOAN_ID,ORIG_DTE,FRST_DTE,Monthly_Rpt_Prd,Loan_Age,Seller_Name,ORIG_RT,ORIG_AMT,Zero_Bal_Code,Delq_Status,ZB_DTE,LPI_DTE
19598,101917807151,01/2018,03/2018,08/01/2019,18.0,"FLAGSTAR BANK, FSB",4.474,85000,NaN,5,,
20835,102021408123,12/2017,02/2018,08/01/2019,19.0,FREEDOM MORTGAGE CORP.,4.125,387000,NaN,5,,
20836,102021408123,12/2017,02/2018,09/01/2019,20.0,FREEDOM MORTGAGE CORP.,4.125,387000,NaN,5,,
20838,102021408123,12/2017,02/2018,11/01/2019,22.0,FREEDOM MORTGAGE CORP.,4.125,387000,NaN,5,,
34776,103423240534,02/2018,04/2018,08/01/2019,17.0,"MOVEMENT MORTGAGE, LLC",4.625,360000,NaN,5,,
...,...,...,...,...,...,...,...,...,...,...,...,...
9319622,998762692998,12/2017,02/2018,11/01/2019,22.0,"CALIBER HOME LOANS, INC.",4.250,231000,NaN,5,,
9319623,998762692998,12/2017,02/2018,12/01/2019,23.0,"CALIBER HOME LOANS, INC.",4.250,231000,NaN,5,,
9321525,998998928844,12/2017,02/2018,12/01/2019,23.0,"UNITED SHORE FINANCIAL SERVICES, LLC DBA UNITE...",3.875,360000,NaN,5,,
9322033,999048069737,12/2017,02/2018,01/01/2019,12.0,TRUIST BANK (FORMERLY SUNTRUST BANK),4.250,190000,NaN,5,,


In [14]:
test.query("LOAN_ID == 102021408123")

,LOAN_ID,ORIG_DTE,FRST_DTE,Monthly_Rpt_Prd,Loan_Age,Seller_Name,ORIG_RT,ORIG_AMT,Zero_Bal_Code,Delq_Status,ZB_DTE,LPI_DTE
20818,102021408123,12/2017,02/2018,03/01/2018,2.0,FREEDOM MORTGAGE CORP.,4.125,387000,NaN,0,,
20819,102021408123,12/2017,02/2018,04/01/2018,3.0,FREEDOM MORTGAGE CORP.,4.125,387000,NaN,0,,
20820,102021408123,12/2017,02/2018,05/01/2018,4.0,FREEDOM MORTGAGE CORP.,4.125,387000,NaN,0,,
20821,102021408123,12/2017,02/2018,06/01/2018,5.0,FREEDOM MORTGAGE CORP.,4.125,387000,NaN,0,,
20822,102021408123,12/2017,02/2018,07/01/2018,6.0,FREEDOM MORTGAGE CORP.,4.125,387000,NaN,0,,
20823,102021408123,12/2017,02/2018,08/01/2018,7.0,FREEDOM MORTGAGE CORP.,4.125,387000,NaN,0,,
20824,102021408123,12/2017,02/2018,09/01/2018,8.0,FREEDOM MORTGAGE CORP.,4.125,387000,NaN,0,,
20825,102021408123,12/2017,02/2018,10/01/2018,9.0,FREEDOM MORTGAGE CORP.,4.125,387000,NaN,0,,
20826,102021408123,12/2017,02/2018,11/01/2018,10.0,FREEDOM MORTGAGE CORP.,4.125,387000,NaN,0,,
20827,102021408123,12/2017,02/2018,12/01/2018,11.0,FREEDOM MORTGAGE CORP.,4.125,387000,NaN,0,,
